# Hackathon

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from prophet import Prophet


In [ ]:
df = pd.read_csv('train/train_data.csv')
df["MEASURED_IN_DATE"] = pd.to_datetime(df["MEASURED_IN_DATE"])
df.info()

In [ ]:
# rename cols for prophet
df.rename(columns={"MEASURED_IN_DATE": "ds", "LIQ_RATE": "y"}, inplace=True)

In [ ]:
# final dataframe for big image 
result = []
# fit every well separately
for i in range(len(df['WELL_NAME'].unique())):
    dff = pd.DataFrame(df.loc[df['WELL_NAME']==df['WELL_NAME'].unique()[i]])

    m = Prophet()

    test_data_size = int(len(dff)*0.2)
    
    m.fit(dff[:-test_data_size]) # todo fix crash???
    
    future = m.make_future_dataframe(periods=test_data_size) #forecasting 365 days in future
    forecast = m.predict(future)

    result.append(pd.DataFrame(forecast))

In [ ]:
plt.gcf().set_size_inches(20,4)
plt.plot(df.y, linewidth = 0.5, color="black")
offset = 0
for gr in result:
    test_data_size = int(len(gr)*0.2)
    plt.plot(gr.index[:int(len(gr)*0.8)]+offset, gr.yhat[:int(len(gr)*0.8)], linestyle = '--', linewidth=0.5, color="red")
    plt.plot(gr.index[int(len(gr)*0.8):]+offset, gr.yhat[int(len(gr)*0.8):], linestyle = '--', linewidth=1, color="green")
    offset += len(gr)
plt.savefig("myplot.png", dpi = 500)

Next i try to fit prophet with all data and then forecast something, which didn't work

In [ ]:
train_stop_name = df['WELL_NAME'][len(df)-int(len(df)*0.2)]
train_stop_index = df.index[df['WELL_NAME'] == train_stop_name][0]
train_stop_index

In [ ]:
test_data_size = len(df)-train_stop_index
test_data_size

In [ ]:
m = Prophet()
m.fit(df[:-test_data_size])

In [ ]:
future = m.make_future_dataframe(periods = 120)
forecast = m.predict(future)

In [ ]:
forecast

In [ ]:
#plt.plot(forecast["ds"].values, forecast["trend"].values)
n=2000
figure = plt.gcf()
figure.set_size_inches(16, 9)
grouped = df.groupby("WELL_NAME")
for name in df.WELL_NAME.unique():
    plt.plot(grouped.get_group(name)["y"])
#np.datetime64(df["ds"].values[0])
#plt.show()

In [ ]:
# Python
def stan_init(m):
    """Retrieve parameters from a trained model.

    Retrieve parameters from a trained model in the format
    used to initialize a new Stan model.

    Parameters
    ----------
    m: A trained model of the Prophet class.

    Returns
    -------
    A Dictionary containing retrieved parameters of m.

    """
    res = {}
    for pname in ['k', 'm', 'sigma_obs']:
        res[pname] = m.params[pname][0][0]
    for pname in ['delta', 'beta']:
        res[pname] = m.params[pname][0]
    return res

df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
df.ds = pd.to_datetime(df.ds)
df1 = df.loc[df['ds'] < '2014-01-19', :]  # All data except the last day
m1 = Prophet(yearly_seasonality=True).fit(df1) # A model fit to all data except the last day



In [ ]:
future = m1.make_future_dataframe(periods=365*2) #forecasting 365 days in future
forecast = m1.predict(future)

In [ ]:
plt.gcf().set_size_inches(20,4)
plt.plot(df1.ds, df1.y)
plt.plot(forecast.ds[len(df1):], forecast.yhat[len(df1):])